## Interacting with Web APIs

In our first class, we examined how to use `curl` to issue requests against web services. We will now see how to achieve the same in Python:

In [ ]:
# We first import the requests library
import requests
url = 'http://freegeoip.net/json/'
resp = requests.get(url)

In [ ]:
# The resp object encapsulates the "response" of the server
# Notice the status code that is displayed. 
# Code 200 means that things went fine
# Code 404 means that the URL was not found
# Codes 5xx mean that something went wrong
resp

In [ ]:
if (resp.status_code == 200):
    print "Everything was ok:", resp.status_code
else:
    print "There was a problem:", resp.status_code

In [ ]:
# Let's see the content of the response
# As you can see, it contain the JSON response
resp.text

In [ ]:
# We want to transform the JSON file into a Python dictionary object
# For that we will use the json library
import json

# The loads (LOADS = LOAD from String) function reads a string that represents a JSON file
data = json.loads(resp.text)

In [ ]:
# Now data is a Python dictionary
# The u'....' characters mean that the string is represented in Unicode
data

In [ ]:
# And we can access the fields of the JSON as we normally access Python dictionary entries
print "Lon:", data["longitude"], "Lat:", data["latitude"]

### Exercise

Read the location of your computer using the GeoIP API, and then use the OpenWeatherMap to query the API and fetch the temperature ((Documentation)[http://openweathermap.org/current#geo]). For this exercise, you will need to:
* Create an account with OpenweatherMap and get an API key
* Study the documentation of the API on OpenWeatherMap
* Learn to read variables from a Web API (freegeoip) and use them as input in another (openweathermap)

In [ ]:
import requests
import json
url = 'http://freegeoip.net/json/'
resp = requests.get(url)
data = json.loads(resp.text)
lon = data["longitude"]
lat = data["latitude"]

#your code here


In [ ]:
url2 = "http://api.openweathermap.org/data/2.5/weather?" + \
     "&lat=" + str(lat) + \
     "&lon=" + str(lon) + \
     "&appid=44db6a862fba0b067b1930da0d769e98" + \
     "&units=imperial"

resp = requests.get(url2)
data = json.loads(resp.text)
data
print data["main"]["temp"]

### Solution for Exercise

In [ ]:
import requests
import json
url = 'http://freegeoip.net/json/'
resp = requests.get(url)
data = json.loads(resp.text)
lon = data["longitude"]
lat = data["latitude"]

# http://api.openweathermap.org/data/2.5/weather?q=New%20York,NY,USA

url2 = 'http://api.openweathermap.org/data/2.5/weather?' + \
    '&lat=' + str(lat) + \
    '&lon=' + str(lon) + \
    '&appid=ffb7b9808e07c9135bdcc7d1e867253d' + \
    '&units=imperial' + \
    '&mode=json'
print url2
resp = requests.get(url2)
data = json.loads(resp.text)
data

## Beyond the basics: Parameters and Headers

The first call that we submitted was very simple. We just fetched a URL, which was represented as a string. However, calling web services by manually concatenating strings to create URLs is not a good practice. It is always better to use a dictionary to pass the parameters to the API call; furthermore, we will often need to pass a set of _headers_ to the API call (as in the case of Mashape).

Let's play a little bit with the FacePlusPlus API, with which we toyed around in our first session:

![Image from NY Times](http://graphics8.nytimes.com/newsgraphics/2016/02/01/iowa-hp/dd8cb1e066b52661f94bb2306fc54189f1c3325e/hp-kk-dem-1.jpg)

In [ ]:
curl --get 
'https://faceplusplus-faceplusplus.p.mashape.com/detection/detect'
'?attribute=glass%2Cpose%2Cgender%2Cage%2Crace%2Csmiling
'&url=http%3A%2F%2Fwww.faceplusplus.com%2Fwp-content%2Fthemes%2Ffaceplusplus%2Fassets%2Fimg%2Fdemo%2F1.jpg' \
  -H 'X-Mashape-Key: zG3wec50exmshxNoF1NMHNRH37GYp1d7oW8jsnWwIMTeMmALxg' \
  -H 'Accept: application/json'

In [ ]:
import requests
import json

facepp_url = "https://faceplusplus-faceplusplus.p.mashape.com/detection/detect"
img_url = "http://previews.123rf.com/images/andresr/andresr1206/andresr120600543/14095175-Happy-group-of-students-smiling-isolated-over-a-white-background--Stock-Photo.jpg"

headers = {
  "X-Mashape-Key": "zG3wec50exmshxNoF1NMHNRH37GYp1d7oW8jsnWwIMTeMmALxg",
  "Accept": "application/json"
}
parameters = {
    'attributes': 'glass,pose,gender,age,race,smiling',
    'url': img_url
}

resp = requests.get(facepp_url, params=parameters, headers=headers, verify=False)
data = json.loads(resp.text)

data

In [ ]:
# We can also pretty print the dictionary object
# print(json.dumps(data, indent=2))

In [ ]:
# And here is a more generic way to pretty print Python data structures, which works for many objects
# import pprint
# pprint.pprint(data)

In [ ]:
len(data["face"])

### Interacting with Alchemy API; POST vs GET

Sometimes, the parameters that we need to pass to the API are too long (e.g., analyzing a piece of long text for sentiment). In such cases, we need to use the "POST" options as opposed to the "GET" function (although admittedly, many APIs will try to be forgiving).

In [ ]:
import requests
import json

url = "http://access.alchemyapi.com/calls/text/TextGetTextSentiment"

# You can register and get your own key
api_key = '3d0b6858f7ef32fdf27ad402f4a9c270c9685d84'

text = '''
If things go as promised, Trump won’t be there Thursday when Fox hosts the final Republican debate before Monday’s Iowa presidential caucuses. He says he’s backing out because of a taunting statement from Fox, though his detractors accuse him of dodging a last showdown with his chief rival, Sen. Ted Cruz (Tex.). Instead, Trump has made plans to materialize elsewhere in Iowa, hosting a benefit for wounded veterans — counter-programming on a ­Trumpian scale of swagger.

His threatened absence from the debate stage is a demonstration of Trump’s perception of his own self-worth, his verifiable status as a ratings-generating gargantuan whose screen persona can translate into millions of advertising dollars. In a sense, it’s an act of subversion by a candidate who has broken all the normal rules of modern campaigns. But it’s also a manifestation of Trump’s philosophy about getting what he wants when he wants it.
'''

headers = {
  "Accept": "application/json"
}

parameters = {
    'outputMode': 'json',
    'apikey' : api_key,
    'text': text,
}

resp = requests.post(url, params=parameters, headers=headers)
data = json.loads(resp.text)

data

And here is a different API call that extracts entities from the text, and also the sentiment for each of these entities. Furthermore, each entity is matched into a "normalized" entry in the Knowledge Graph.

In [7]:
import requests
import json

url = "http://gateway-a.watsonplatform.net/calls/url/URLGetRankedNamedEntities"


# You can register and get your own key
api_key = '3d0b6858f7ef32fdf27ad402f4a9c270c9685d84'


text_url = 'https://www.washingtonpost.com/politics/question-whats-clintons-message-answer-all-of-the-above/2016/02/06/5df28fcc-cc4a-11e5-88ff-e2d1b4289c2f_story.html'
headers = {
  "Accept": "application/json"
}

parameters = {
    'outputMode': 'json',
    'apikey' : api_key,
     'sentiment' :1,
    'knowledgeGraph': 1,
    'url': text_url
}

resp = requests.post(url, params=parameters, headers=headers)
data = json.loads(resp.text)


{u'status': u'OK', u'totalTransactions': u'3', u'language': u'english', u'url': u'https://www.washingtonpost.com/politics/question-whats-clintons-message-answer-all-of-the-above/2016/02/06/5df28fcc-cc4a-11e5-88ff-e2d1b4289c2f_story.html', u'entities': [{u'count': u'15', u'knowledgeGraph': {u'typeHierarchy': u'/people/politicians/democrats/hillary clinton'}, u'sentiment': {u'mixed': u'1', u'score': u'-0.284936', u'type': u'negative'}, u'text': u'Hillary Clinton', u'relevance': u'0.808725', u'type': u'Person', u'disambiguated': {u'website': u'http://www.state.gov/secretary/index.htm', u'yago': u'http://yago-knowledge.org/resource/Hillary_Rodham_Clinton', u'name': u'Hillary Rodham Clinton', u'freebase': u'http://rdf.freebase.com/ns/m.0d06m5', u'opencyc': u'http://sw.opencyc.org/concept/Mx4rvV7SqpwpEbGdrcN5Y29ycA', u'subType': [u'Politician', u'Appointee', u'AwardWinner', u'BoardMember', u'Celebrity', u'HallOfFameInductee', u'OperaCharacter', u'Senator', u'U.S.Congressperson', u'TVActor'],

In [9]:

print data["entities"]

[{u'count': u'15', u'knowledgeGraph': {u'typeHierarchy': u'/people/politicians/democrats/hillary clinton'}, u'sentiment': {u'mixed': u'1', u'score': u'-0.284936', u'type': u'negative'}, u'text': u'Hillary Clinton', u'relevance': u'0.808725', u'type': u'Person', u'disambiguated': {u'website': u'http://www.state.gov/secretary/index.htm', u'yago': u'http://yago-knowledge.org/resource/Hillary_Rodham_Clinton', u'name': u'Hillary Rodham Clinton', u'freebase': u'http://rdf.freebase.com/ns/m.0d06m5', u'opencyc': u'http://sw.opencyc.org/concept/Mx4rvV7SqpwpEbGdrcN5Y29ycA', u'subType': [u'Politician', u'Appointee', u'AwardWinner', u'BoardMember', u'Celebrity', u'HallOfFameInductee', u'OperaCharacter', u'Senator', u'U.S.Congressperson', u'TVActor'], u'dbpedia': u'http://dbpedia.org/resource/Hillary_Rodham_Clinton'}}, {u'count': u'12', u'knowledgeGraph': {u'typeHierarchy': u'/senators/bernie sanders/sen. bernie sanders'}, u'sentiment': {u'mixed': u'1', u'score': u'-0.190996', u'type': u'negative'}

In [10]:
for person in data["entities"]:
    print person["text"], person["relevance"]

Hillary Clinton 0.808725
Sen. Bernie Sanders 0.595727
Clinton 0.317174
Bill Clinton 0.291593
President Obama 0.238528
Congress 0.23596
N.H. 0.200827
Derry 0.186155
Chuck Todd 0.183055
Iowa 0.175775
CONCORD 0.174566
Mich. 0.173232
presidential debate 0.170401
Manchester 0.170362
Nick Pangaro 0.169592
Washington Post 0.168052
clean energy 0.165608
President 0.165495
Rep. Tim Roemer 0.165038
Joel Benenson 0.164558
Vermont 0.160294
Brian Fallon 0.157914
MSNBC 0.154535
foreign policy 0.153038
ISIS 0.145159
Abby Phillip 0.144621
New Hampshire primary 0.140428
New Hampshire 0.139507
consultant 0.136629
Flint 0.132448
Boys and Girls Club 0.13159
Durham 0.130463
Russia 0.130272
Medicare 0.129666
Mika Brzyznski 0.127369
insurance system 0.126433
Iran 0.118187
tumulty@washpost.com 0.118187
gearan@washpost.com 0.118187


In [ ]:
import pandas as pd
from pandas.io.json import json_normalize
df = json_normalize(data["entities"])
df